# 📊 **Performance of ReAct Agent Running a Sample Question from TriviaQA Benchmark**


In [12]:
import os
import openai
from langchain_community.chat_models import ChatOpenAI
from langchain_core.language_models.chat_models import BaseChatModel
from agential.cog.agent.react import ReActAgent
from agential.cog.prompts.react import (
    TRIVIAQA_FEWSHOT_EXAMPLES,
    REACT_INSTRUCTION_TRIVIAQA,
)
print("Environment has been set up")

Environment has been set up


In [16]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"API key has been activated: {openai.api_key}")

gpt_turbo = ChatOpenAI(OPENAI_API_KEY="213")

print("Initializing the agent")
agent = ReActAgent(llm=gpt_turbo)

question = "American Callan Pinckney's eponymously named system became a best-selling (1980s-2000s) book/video franchise in what genre?"
outputs = agent.generate(question=question, examples=TRIVIAQA_FEWSHOT_EXAMPLES,
        prompt=REACT_INSTRUCTION_TRIVIAQA)

print(f"Question: {question}") 

if outputs:
    for output in outputs:
        print("Thought:", output.thought)
        print("Action:", output.action)
        print("Observation:", output.observation)
else:
    print("No output generated.")

WARNING! OPENAI_API_KEY is not default parameter.
                    OPENAI_API_KEY was transferred to model_kwargs.
                    Please confirm that OPENAI_API_KEY is what you intended.


API key has been activated: None


ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)